In [1]:
import pandas as pd
import glob
import re

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
psm_files = glob.glob("MSFragger_Output/*_psm.tsv")
psm_files

['MSFragger_Output\\acidorvorans_psm.tsv',
 'MSFragger_Output\\acidovorans_psm.tsv',
 'MSFragger_Output\\aurantiaca_psm.tsv',
 'MSFragger_Output\\baltica_psm.tsv',
 'MSFragger_Output\\cereus_psm.tsv',
 'MSFragger_Output\\comes_psm.tsv',
 'MSFragger_Output\\cryptum_psm.tsv',
 'MSFragger_Output\\fragilis_psm.tsv',
 'MSFragger_Output\\gilvus_psm.tsv',
 'MSFragger_Output\\gnavus_psm.tsv',
 'MSFragger_Output\\griseorubens_psm.tsv',
 'MSFragger_Output\\ljungdahlii_psm.tsv',
 'MSFragger_Output\\necator_psm.tsv',
 'MSFragger_Output\\palustris_psm.tsv',
 'MSFragger_Output\\subtilis_psm.tsv',
 'MSFragger_Output\\thermosulf_psm.tsv',
 'MSFragger_Output\\thet_psm.tsv']

In [4]:
run_files = glob.glob("MSFragger_Output/*_run.tsv")
run_files

['MSFragger_Output\\acidovorans_01_run.tsv',
 'MSFragger_Output\\acidovorans_02_run.tsv',
 'MSFragger_Output\\acidovorans_03_run.tsv',
 'MSFragger_Output\\aurantiaca_01_run.tsv',
 'MSFragger_Output\\aurantiaca_02_run.tsv',
 'MSFragger_Output\\aurantiaca_03_run.tsv',
 'MSFragger_Output\\baltica_01C_run.tsv',
 'MSFragger_Output\\baltica_01Inf_run.tsv',
 'MSFragger_Output\\baltica_02C_run.tsv',
 'MSFragger_Output\\baltica_02Inf_run.tsv',
 'MSFragger_Output\\baltica_03C_run.tsv',
 'MSFragger_Output\\baltica_03Inf_run.tsv',
 'MSFragger_Output\\cereus_01_run.tsv',
 'MSFragger_Output\\cereus_02_run.tsv',
 'MSFragger_Output\\cereus_03_run.tsv',
 'MSFragger_Output\\comes_Carb_run.tsv',
 'MSFragger_Output\\comes_Glc_run.tsv',
 'MSFragger_Output\\comes_LIB_run.tsv',
 'MSFragger_Output\\cryptum_01_run.tsv',
 'MSFragger_Output\\cryptum_02_run.tsv',
 'MSFragger_Output\\cryptum_03_run.tsv',
 'MSFragger_Output\\fragilis_01_run.tsv',
 'MSFragger_Output\\fragilis_Carb_01_run.tsv',
 'MSFragger_Output\\fr

In [5]:
def extract_scannum(Spectrum):
    pattern = r'\.\d*\.'
    lst = []
    for s in Spectrum:
        m = re.search(pattern, s)
        if m:
            m = m[0]
            m = m.replace('.', '')
            m = int(m)
            lst.append(m)
        else:
            lst.append(None)
    return (lst)

In [6]:
def combine_run_files(files):
    dfs = []
    for file in files:
        name = re.split('_run.tsv', file)
        name = name[0]
        pattern = r"\\"
        name = re.split(pattern, name)
        name = name[1]
        name = re.split('_', name)
        name = name[0]
        df = pd.read_csv(file, sep = '\t')
        keeps = ['scannum', 'best_locs', 'peptide']
        df = df[df.best_locs.notnull()]
        df = df[keeps]
        df['Organism'] = [name] * len(df)
        df = df.rename(columns = {'peptide': 'Peptide'})
        dfs.append(df)
    dfs = pd.concat(dfs)
    return (dfs)


In [7]:
def combine_psm_files(files):
    dfs = []
    i = 0
    for file in files:
        name = re.split('_psm.tsv', file)
        name = name[0]
        pattern = r"\\"
        name = re.split(pattern, name)
        name = name[1]
        keeps = ['scannum', 'Peptide', 'Observed Modifications', 'Protein ID', 'Gene']
        df = pd.read_csv(file, sep = '\t')   
        df = df[df['PeptideProphet Probability'] >=0.99]
        df = df[df['Observed Modifications'].notnull()]
        df = df[df.Gene.notnull()]
        df = df[~df.Gene.str.contains('Daci')]
        df['scannum'] = extract_scannum(df.Spectrum)
        df = df[keeps]
        df['Organism'] = [name] * len(df)        
        dfs.append(df)
    df = pd.concat(dfs)
    return(df)

In [8]:
combinded_psm_df = combine_psm_files(psm_files)
combined_run_df = combine_run_files(run_files)

C:\Users\nanel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\nanel\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [9]:
df = pd.merge(combinded_psm_df, combined_run_df, how='inner')
df

,scannum,Peptide,Observed Modifications,Protein ID,Gene,Organism,best_locs
0,1982,AAASSTAAAKTPAKR,"Asn->Gln(14.015650), Asp->Glu(14.015650), Gly-...",A9BMX9,rpoD,acidovorans,AAASSTAAAkTPAKR
1,2020,AAASSTAAAKTPAKR,"Ala->Val(28.031300), Cys->Met(28.031300), Delt...",A9BMX9,rpoD,acidovorans,AAASSTAAAktpakR
2,2040,AASAAAPAAGKKPAAK,"Ala->Val(28.031300), Cys->Met(28.031300), Delt...",A9BWE0,rsfS,acidovorans,AASAAAPAAGkkpaaK
3,2046,KPMVGNNVSHANNK,"Ala->Ser(15.994915), Oxidation(15.994915), Phe...",A9BNU9,rpmB,acidovorans,KPmVGNNVSHANNK
4,2148,AAASSTAAAKTPAK,"Asn->Gln(14.015650), Asp->Glu(14.015650), Gly-...",A9BMX9,rpoD,acidovorans,AAASSTAAAktPAK
...,...,...,...,...,...,...,...
142141,53487,MVPPPLAFEVLDAVMEKLPGFPIVLHGSSSVPEEEVETINK,Tyr->Met(-32.022844),Q8A734,BT_1691,thet,MVPPPLAFEVLDavmeklPGFPIVLHGSSSVPEEEVETINK
142142,53502,NMVTGAAQMDGAIIVCAATDGPMPQTR,Xle->Gly(-56.062600),Q8A463,tuf,thet,NMVTGAAQMDGAIivcaATDGPMPQTR
142143,53503,CFFIELSEIITGK,"Ammonia-loss(-17.026549), Gln->pyro-Glu(-17.02...",Q8A017,BT_4204,thet,cfFIELSEIITGK
142144,53516,QEVAQDIINHLVAVLTH,"Ammonia-loss(-17.026549), Gln->pyro-Glu(-17.02...",Q8A812,coaBC,thet,qevAQDIINHLVAVLTH


In [10]:
df.iloc[0:2]

,scannum,Peptide,Observed Modifications,Protein ID,Gene,Organism,best_locs
0,1982,AAASSTAAAKTPAKR,"Asn->Gln(14.015650), Asp->Glu(14.015650), Gly-...",A9BMX9,rpoD,acidovorans,AAASSTAAAkTPAKR
1,2020,AAASSTAAAKTPAKR,"Ala->Val(28.031300), Cys->Met(28.031300), Delt...",A9BMX9,rpoD,acidovorans,AAASSTAAAktpakR


In [11]:
list(df.iloc[0:2]['Observed Modifications'])

['Asn->Gln(14.015650), Asp->Glu(14.015650), Gly->Ala(14.015650), Methyl(14.015650), Ser->Thr(14.015650), Val->Xle(14.015650)',
 'Ala->Val(28.031300), Cys->Met(28.031300), Delta:H(4)C(2)(28.031300), Dimethyl(28.031300), Ethyl(28.031300)']

In [12]:
genes = df[['Gene', 'Organism']]
genes = genes.drop_duplicates()
genes = genes.groupby('Gene').count()
genes = pd.DataFrame(genes)
genes = genes[genes.Organism > 5]
genes = genes.index
homologous_genes = df[['Gene', 'Organism']]
homologous_genes = homologous_genes.drop_duplicates()
homologous_genes = homologous_genes[homologous_genes.Gene.isin(genes)]
homologous_genes.to_csv('Homologous_genes.csv')

In [13]:
len(pd.unique(homologous_genes.Gene))

218